In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Load data

In [3]:
df_d = pd.read_csv("../data/details/details_2000.csv.gz")
df_f = pd.read_csv("../data/fatalities/fatalities_2000.csv.gz")
df_l = pd.read_csv("../data/locations/locations_2000.csv.gz")

In [4]:
# Merge details + fatalities (many fatalities per event possible)
df_year = df_d.merge(df_f, on="EVENT_ID", how="left")

# Merge with locations (many locations per event)
df_year = df_year.merge(df_l, on="EVENT_ID", how="left")

From the documentation, each episode can have multiple events related to it. This means 'EPISODE_ID' is a broader classification and 'EVENT_ID' is the true index with all unique values. So when we merge on 'EVENT_ID' pandas creates two colums 'EPISODE_ID_x' and 'EPISODE_ID_y' from df_d and df_l. Both these columns are equivalent, so we can safely drop one and rename the other back to 'EPISODE_ID'.

In [102]:
df_year = df_year.rename(columns={'EPISODE_ID_x': 'EPISODE_ID'}).drop(columns=['EPISODE_ID_y'])

# Remove irrelevant columns

In [103]:
cols_to_remove = ['SOURCE', 'DATA_SOURCE', 'CATEGORY','EPISODE_NARRATIVE', 'EVENT_NARRATIVE','STATE_FIPS','CZ_FIPS','TOR_OTHER_CZ_NAME', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_STATE','WFO','TOR_OTHER_WFO']

In [104]:
df_year.drop(columns=cols_to_remove, errors='ignore', inplace=True)

# Create column categories

## ID columns

In [105]:
import re

In [106]:
ID_cols = [col for col in df_year.columns if re.search(r'_ID$', col.upper())]
ID_cols

['EPISODE_ID', 'EVENT_ID', 'FATALITY_ID']

In [107]:
df_year[ID_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53934 entries, 0 to 53933
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   EPISODE_ID   53934 non-null  int64  
 1   EVENT_ID     53934 non-null  int64  
 2   FATALITY_ID  520 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 1.2 MB


In [108]:
for col in ID_cols:
    df_year[col] = df_year[col].astype('category')


In [109]:
df_year[ID_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53934 entries, 0 to 53933
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   EPISODE_ID   53934 non-null  category
 1   EVENT_ID     53934 non-null  category
 2   FATALITY_ID  520 non-null    category
dtypes: category(3)
memory usage: 3.5 MB


In [110]:
df_year[ID_cols]

,EPISODE_ID,EVENT_ID,FATALITY_ID
0,1104812,5165377,NaN
1,1104812,5165378,NaN
2,1104812,5165379,NaN
3,1105342,5165449,NaN
4,1101140,5172568,NaN
...,...,...,...
53929,2414768,5126692,NaN
53930,2414731,5127563,NaN
53931,2414804,5127165,NaN
53932,2414804,5127830,NaN


# Timing columns

In [40]:
timing_cols = [col for col in df_year.columns 
               if any(key in col.upper() for key in ['YEAR', 'DATE', 'TIME', 'MONTH', 'DAY'])]
timing_cols

['BEGIN_YEARMONTH',
 'BEGIN_DAY',
 'BEGIN_TIME',
 'END_YEARMONTH',
 'END_DAY',
 'END_TIME',
 'YEAR',
 'MONTH_NAME',
 'BEGIN_DATE_TIME',
 'CZ_TIMEZONE',
 'END_DATE_TIME',
 'FAT_YEARMONTH',
 'FAT_DAY',
 'FAT_TIME',
 'FATALITY_DATE',
 'EVENT_YEARMONTH',
 'YEARMONTH']

In [41]:
df_year[timing_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53934 entries, 0 to 53933
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  53934 non-null  int64  
 1   BEGIN_DAY        53934 non-null  int64  
 2   BEGIN_TIME       53934 non-null  int64  
 3   END_YEARMONTH    53934 non-null  int64  
 4   END_DAY          53934 non-null  int64  
 5   END_TIME         53934 non-null  int64  
 6   YEAR             53934 non-null  int64  
 7   MONTH_NAME       53934 non-null  object 
 8   BEGIN_DATE_TIME  53934 non-null  object 
 9   CZ_TIMEZONE      53934 non-null  object 
 10  END_DATE_TIME    53934 non-null  object 
 11  FAT_YEARMONTH    520 non-null    float64
 12  FAT_DAY          520 non-null    float64
 13  FAT_TIME         520 non-null    float64
 14  FATALITY_DATE    520 non-null    object 
 15  EVENT_YEARMONTH  520 non-null    float64
 16  YEARMONTH        53934 non-null  int64  
dtypes: float64(4

In [42]:
df_year[timing_cols].head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,YEAR,MONTH_NAME,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_DATE,EVENT_YEARMONTH,YEARMONTH
0,200012,31,600,200012,31,900,2000,December,31-DEC-00 06:00:00,EST,31-DEC-00 09:00:00,NaN,NaN,NaN,NaN,NaN,200012
1,200012,31,600,200012,31,900,2000,December,31-DEC-00 06:00:00,EST,31-DEC-00 09:00:00,NaN,NaN,NaN,NaN,NaN,200012
2,200012,31,700,200012,31,800,2000,December,31-DEC-00 07:00:00,EST,31-DEC-00 08:00:00,NaN,NaN,NaN,NaN,NaN,200012
3,200012,13,2200,200012,14,400,2000,December,13-DEC-00 22:00:00,EST,14-DEC-00 04:00:00,NaN,NaN,NaN,NaN,NaN,200012
4,200008,3,1410,200008,3,1410,2000,August,03-AUG-00 14:10:00,CST,03-AUG-00 14:10:00,NaN,NaN,NaN,NaN,NaN,200008


In [43]:
'YEAR' in df_d.columns

True

In [44]:
'YEAR' in df_f.columns

False

In [45]:
'YEAR' in df_l.columns

False

In [46]:
[c for c in list(df_d.columns) if 'MONTH' in c]

['BEGIN_YEARMONTH', 'END_YEARMONTH', 'MONTH_NAME']

In [47]:
df_year[['BEGIN_YEARMONTH', 'END_YEARMONTH', 'YEAR','MONTH_NAME']]

,BEGIN_YEARMONTH,END_YEARMONTH,YEAR,MONTH_NAME
0,200012,200012,2000,December
1,200012,200012,2000,December
2,200012,200012,2000,December
3,200012,200012,2000,December
4,200008,200008,2000,August
...,...,...,...,...
53929,200001,200001,2000,January
53930,200001,200001,2000,January
53931,200002,200002,2000,February
53932,200002,200002,2000,February


## Remove redundencies from year, month

In [48]:
df_year['MONTH'] = df_year['BEGIN_YEARMONTH'].astype(str).str[-2:].astype(int)

In [49]:
cols_to_drop = ['FAT_YEARMONTH', 'EVENT_YEARMONTH', 'YEARMONTH', 'BEGIN_YEARMONTH', 'END_YEARMONTH']
df_year.drop(columns=cols_to_drop, inplace=True)

# Remove them from timing_cols
timing_cols = [col for col in timing_cols if col not in cols_to_drop]

In [50]:
timing_cols.append('MONTH')

In [51]:
df_year[df_year['FAT_DAY'].notna()][timing_cols].head()

,BEGIN_DAY,BEGIN_TIME,END_DAY,END_TIME,YEAR,MONTH_NAME,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,FAT_DAY,FAT_TIME,FATALITY_DATE,MONTH
128,28,1200,28,1200,2000,January,28-JAN-00 12:00:00,CST,28-JAN-00 12:00:00,28.0,0.0,01/28/2000 12:00:00,1
132,25,0,28,0,2000,January,25-JAN-00 00:00:00,CST,28-JAN-00 00:00:00,28.0,0.0,01/28/2000 12:00:00,1
333,25,0,28,0,2000,January,25-JAN-00 00:00:00,CST,28-JAN-00 00:00:00,27.0,0.0,01/27/2000 12:00:00,1
537,14,100,14,900,2000,January,14-JAN-00 01:00:00,EST,14-JAN-00 09:00:00,14.0,0.0,01/14/2000 12:00:00,1
538,14,100,14,900,2000,January,14-JAN-00 01:00:00,EST,14-JAN-00 09:00:00,14.0,0.0,01/14/2000 12:00:00,1


In [52]:
df_year[timing_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53934 entries, 0 to 53933
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_DAY        53934 non-null  int64  
 1   BEGIN_TIME       53934 non-null  int64  
 2   END_DAY          53934 non-null  int64  
 3   END_TIME         53934 non-null  int64  
 4   YEAR             53934 non-null  int64  
 5   MONTH_NAME       53934 non-null  object 
 6   BEGIN_DATE_TIME  53934 non-null  object 
 7   CZ_TIMEZONE      53934 non-null  object 
 8   END_DATE_TIME    53934 non-null  object 
 9   FAT_DAY          520 non-null    float64
 10  FAT_TIME         520 non-null    float64
 11  FATALITY_DATE    520 non-null    object 
 12  MONTH            53934 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 5.3+ MB


In [53]:
# For day-level analysis drop the time columns
time_cols = [col for col in timing_cols if '_TIME' in col]
df_year.drop(columns=time_cols, inplace=True)
timing_cols = [col for col in timing_cols if col not in time_cols]

df_year['DURATION_DAYS'] = df_year['END_DAY'] - df_year['BEGIN_DAY'] + 1
timing_cols.append('DURATION_DAYS')

In [54]:
df_year[timing_cols]

,BEGIN_DAY,END_DAY,YEAR,MONTH_NAME,FAT_DAY,FATALITY_DATE,MONTH,DURATION_DAYS
0,31,31,2000,December,NaN,NaN,12,1
1,31,31,2000,December,NaN,NaN,12,1
2,31,31,2000,December,NaN,NaN,12,1
3,13,14,2000,December,NaN,NaN,12,2
4,3,3,2000,August,NaN,NaN,8,1
...,...,...,...,...,...,...,...,...
53929,11,11,2000,January,NaN,NaN,1,1
53930,25,25,2000,January,NaN,NaN,1,1
53931,1,29,2000,February,NaN,NaN,2,29
53932,1,29,2000,February,NaN,NaN,2,29


In [55]:
df_d[['BEGIN_YEARMONTH','BEGIN_DATE_TIME','END_DATE_TIME']]

,BEGIN_YEARMONTH,BEGIN_DATE_TIME,END_DATE_TIME
0,200012,31-DEC-00 06:00:00,31-DEC-00 09:00:00
1,200012,31-DEC-00 06:00:00,31-DEC-00 09:00:00
2,200012,31-DEC-00 07:00:00,31-DEC-00 08:00:00
3,200012,13-DEC-00 22:00:00,14-DEC-00 04:00:00
4,200008,03-AUG-00 14:10:00,03-AUG-00 14:10:00
...,...,...,...
52002,200001,11-JAN-00 05:00:00,11-JAN-00 09:00:00
52003,200001,25-JAN-00 10:30:00,25-JAN-00 10:30:00
52004,200002,01-FEB-00 00:00:00,29-FEB-00 23:59:00
52005,200002,01-FEB-00 00:00:00,29-FEB-00 23:59:00


In [4]:
import calendar
def clean_timing_columns(df):
    """
    Cleans timing columns in NOAA storm events DataFrame.
    Returns a DataFrame with:
    - YEAR, BEGIN_MONTH, END_MONTH, BEGIN_MONTH_NAME
    - BEGIN_DAY, END_DAY, FAT_DAY
    - DURATION_DAYS (computed from datetime)
    Drops redundant _YEARMONTH and _TIME columns.
    """

    df = df.copy()

    # YEAR
    df['YEAR'] = df['YEAR'].astype(int)
    
    # Create BEGIN_MONTH and END_MONTH from BEGIN_YEARMONTH, END_YEARMONTH columns 
    df['BEGIN_MONTH']=df_year['BEGIN_YEARMONTH'].astype(str).str[-2:].astype(int)
    df['END_MONTH']=df_year['END_YEARMONTH'].astype(str).str[-2:].astype(int)

    # Create BEGIN_MONTH_NAME categorical
    df['BEGIN_MONTH_NAME'] = df['BEGIN_MONTH'].apply(lambda x: calendar.month_abbr[x])
    df['BEGIN_MONTH_NAME'] = pd.Categorical(
        df['BEGIN_MONTH_NAME'],
        categories=list(calendar.month_abbr)[1:],  # Jan→Dec
        ordered=True
    )
 
    # Compute DURATION_DAYS using datetime columns
    # parse datetimes
    df['BEGIN_DATE_TIME'] = pd.to_datetime(df['BEGIN_DATE_TIME'], errors='coerce')
    df['END_DATE_TIME']   = pd.to_datetime(df['END_DATE_TIME'], errors='coerce')

    # duration (cross-month handled correctly)
    bd = df['BEGIN_DATE_TIME'].dt.floor('D')
    ed = df['END_DATE_TIME'].dt.floor('D')
    df['DURATION_DAYS'] = (ed - bd).dt.days + 1

    # FAT_DAY to Int64 (nullable integer)
    # df['FAT_DAY'] = pd.to_numeric(df['FAT_DAY'], errors='coerce').astype('Int64')
    df['FAT_DAY'] = df['FAT_DAY'].astype('Int64')

    # drop unused timing columns
    drop_cols = [c for c in df.columns if any(key in c.upper() for key in ['YEARMONTH', '_TIME','_DATE'])]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    df.drop(columns=['MONTH_NAME'], inplace=True, errors='ignore')  # redundant with BEGIN_MONTH_NAME

    return df

In [5]:
df_clean_time = clean_timing_columns(df_year)

In [6]:
timing_cols = [col for col in df_clean_time.columns 
               if any(key in col.upper() for key in ['YEAR', 'DATE', 'TIME', 'MONTH', 'DAY'])]
timing_cols

['BEGIN_DAY',
 'END_DAY',
 'YEAR',
 'FAT_DAY',
 'BEGIN_MONTH',
 'END_MONTH',
 'BEGIN_MONTH_NAME',
 'DURATION_DAYS']

In [7]:
df_clean_time[timing_cols]

,BEGIN_DAY,END_DAY,YEAR,FAT_DAY,BEGIN_MONTH,END_MONTH,BEGIN_MONTH_NAME,DURATION_DAYS
0,31,31,2000,<NA>,12,12,Dec,1
1,31,31,2000,<NA>,12,12,Dec,1
2,31,31,2000,<NA>,12,12,Dec,1
3,13,14,2000,<NA>,12,12,Dec,2
4,3,3,2000,<NA>,8,8,Aug,1
...,...,...,...,...,...,...,...,...
53929,11,11,2000,<NA>,1,1,Jan,1
53930,25,25,2000,<NA>,1,1,Jan,1
53931,1,29,2000,<NA>,2,2,Feb,29
53932,1,29,2000,<NA>,2,2,Feb,29


In [89]:
df_clean_time[df_clean_time['FAT_DAY'].notna()][timing_cols].info()

<class 'pandas.core.frame.DataFrame'>
Index: 520 entries, 128 to 53880
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   BEGIN_DAY         520 non-null    int64   
 1   END_DAY           520 non-null    int64   
 2   YEAR              520 non-null    int64   
 3   FAT_DAY           520 non-null    Int64   
 4   BEGIN_MONTH       520 non-null    int64   
 5   END_MONTH         520 non-null    int64   
 6   BEGIN_MONTH_NAME  520 non-null    category
 7   DURATION_DAYS     520 non-null    int64   
dtypes: Int64(1), category(1), int64(6)
memory usage: 33.9 KB


In [5]:
import sys
import os

# Add the src folder to Python path
sys.path.append(os.path.abspath("../src"))
from data_cleaner import drop_unwanted_cols, clean_id_cols, clean_timing_cols

In [6]:
df_year.shape

(53934, 71)

In [7]:
drop_unwanted_cols(df_year)

In [8]:
df_year.shape

(53934, 59)

In [9]:
clean_id_cols(df_year)

In [10]:
df_year[['EPISODE_ID','EVENT_ID', 'FATALITY_ID']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53934 entries, 0 to 53933
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   EPISODE_ID   53934 non-null  category
 1   EVENT_ID     53934 non-null  category
 2   FATALITY_ID  520 non-null    category
dtypes: category(3)
memory usage: 3.5 MB


In [11]:
df_year.shape

(53934, 58)

In [12]:
clean_timing_cols(df_year)

In [13]:
df_year.shape

(53934, 49)

In [14]:
df_year.head()

,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,EVENT_TYPE,CZ_TYPE,CZ_NAME,INJURIES_DIRECT,...,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2,BEGIN_MONTH,END_MONTH,BEGIN_MONTH_NAME,DURATION_DAYS
0,31,31,1104812,5165377,FLORIDA,2000,Extreme Cold/Wind Chill,Z,INLAND PALM BEACH,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,Dec,1
1,31,31,1104812,5165378,FLORIDA,2000,Extreme Cold/Wind Chill,Z,INLAND COLLIER,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,Dec,1
2,31,31,1104812,5165379,FLORIDA,2000,Extreme Cold/Wind Chill,Z,INLAND DADE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,Dec,1
3,13,14,1105342,5165449,WEST VIRGINIA,2000,Winter Storm,Z,PRESTON,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,Dec,2
4,3,3,1101140,5172568,MISSISSIPPI,2000,Thunderstorm Wind,C,SCOTT,0,...,NaN,FORKVILLE,32.45,-89.65,NaN,NaN,8,8,Aug,1
